In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, cohen_kappa_score, roc_auc_score, matthews_corrcoef, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

In [ ]:
#method Ⅱ  u-test
from scipy.stats import mannwhitneyu

def perform_mannwhitneyu_test(sample1, sample2):
    statistic, p_value = mannwhitneyu(sample1, sample2, alternative='two-sided')
    return p_value

significant_results = []

for i in range(postive_filtered_delta_pcc_pair_df.shape[0]):
    # Mann-Whitney U
    p_value = perform_mannwhitneyu_test(postive_filtered_delta_pcc_pair_df.iloc[i].values, negative_filtered_delta_pcc_pair_df.iloc[i].values)
    
    # 
    if p_value < 0.05:
        significant_results.append((i, p_value))

for result in significant_results:
    print(f"Row {result[0]}: P-value = {result[1]}")


In [3]:
# Set directory
os.chdir('path')
os.getcwd()

'/home/wang/TCGA/mRNA'

In [4]:
# Load data
train = pd.read_csv("path_to_mRNA_data/input.csv", index_col=0)
target = pd.read_csv("path_to_labels/label.csv", index_col=0).loc[train.columns, :].values.ravel()
indexs = train.index
train = train.T.values
print("train.shape:", train.shape)

train.shape: (180, 100)


In [ ]:
#LASSO selection - find best lasso value
#lasso_cv = LassoCV(alphas=np.logspace(-3, -2, 30), cv=5, random_state=random_states[i])
#lasso_cv.fit(X_standard, target)
#print(f"Selected alpha in iteration {i+1}: {lasso_cv.alpha_}")  # Print selected alpha value

In [ ]:
# Parameters
loopn = 10  # number of repetitions
n_folds = 10  # 10-fold cross-validation

# Store performance metrics across runs for training and testing sets
train_sensitivity_list, train_specificity_list, train_accuracy_list, train_f1_list, train_kappa_list, train_auc_list, train_mcc_list = [], [], [], [], [], [], []
test_sensitivity_list, test_specificity_list, test_accuracy_list, test_f1_list, test_kappa_list, test_auc_list, test_mcc_list = [], [], [], [], [], [], []

# Perform feature scaling on the entire dataset
standardScaler = StandardScaler()
X_standard = standardScaler.fit_transform(train) 

# Perform LASSO feature selection on the entire dataset
lasso = Lasso(alpha=0.01, random_state=42)
lasso.fit(X_standard, target)
lasso_feature_indices = np.abs(lasso.coef_) > 0
lasso_X = X_standard[:, lasso_feature_indices]
selected_features = np.array(indexs)[lasso_feature_indices]

# Run loop for each repetition
np.random.seed(10)
random_states = np.random.choice(range(101), loopn, replace=False)

for i in tqdm(range(loopn)):
    X_train, X_test, y_train, y_test = train_test_split(lasso_X, target, test_size=0.4, random_state=random_states[i])

    # Initialize logistic regression model
    logistic_reg_model = LogisticRegression()

    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_states[i])

    fold_train_sensitivity, fold_train_specificity, fold_train_accuracy, fold_train_f1, fold_train_kappa, fold_train_auc, fold_train_mcc = [], [], [], [], [], [], []

    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Train the model on the current fold
        logistic_reg_model.fit(X_train_fold, y_train_fold)

        # Predict on the validation set of the current fold
        y_pred_val = logistic_reg_model.predict(X_val_fold)
        y_prob_val = logistic_reg_model.predict_proba(X_val_fold)[:, 1]

        # Calculate confusion matrix for the current fold on the training set
        tn, fp, fn, tp = confusion_matrix(y_val_fold, y_pred_val).ravel()
        fold_train_sensitivity.append(tp / (tp + fn))
        fold_train_specificity.append(tn / (tn + fp))
        fold_train_accuracy.append(accuracy_score(y_val_fold, y_pred_val))
        fold_train_f1.append(f1_score(y_val_fold, y_pred_val))
        fold_train_kappa.append(cohen_kappa_score(y_val_fold, y_pred_val))
        fold_train_auc.append(roc_auc_score(y_val_fold, y_prob_val))
        fold_train_mcc.append(matthews_corrcoef(y_val_fold, y_pred_val))

    # Average performance metrics across folds on the training set for the current run
    train_sensitivity_list.append(np.mean(fold_train_sensitivity))
    train_specificity_list.append(np.mean(fold_train_specificity))
    train_accuracy_list.append(np.mean(fold_train_accuracy))
    train_f1_list.append(np.mean(fold_train_f1))
    train_kappa_list.append(np.mean(fold_train_kappa))
    train_auc_list.append(np.mean(fold_train_auc))
    train_mcc_list.append(np.mean(fold_train_mcc))

    # Evaluate the model on the independent test set
    y_pred_test = logistic_reg_model.predict(X_test)
    y_prob_test = logistic_reg_model.predict_proba(X_test)[:, 1]
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()

    test_sensitivity_list.append(tp / (tp + fn))
    test_specificity_list.append(tn / (tn + fp))
    test_accuracy_list.append(accuracy_score(y_test, y_pred_test))
    test_f1_list.append(f1_score(y_test, y_pred_test))
    test_kappa_list.append(cohen_kappa_score(y_test, y_pred_test))
    test_auc_list.append(roc_auc_score(y_test, y_prob_test))
    test_mcc_list.append(matthews_corrcoef(y_test, y_pred_test))

avg_train_sensitivity = np.mean(train_sensitivity_list)
avg_train_specificity = np.mean(train_specificity_list)
avg_train_accuracy = np.mean(train_accuracy_list)
avg_train_f1 = np.mean(train_f1_list)
avg_train_kappa = np.mean(train_kappa_list)
avg_train_auc = np.mean(train_auc_list)
avg_train_mcc = np.mean(train_mcc_list)

print("Training Set Metrics:")
print(f"Avg Sensitivity: {avg_train_sensitivity:.4f}")
print(f"Avg Specificity: {avg_train_specificity:.4f}")
print(f"Avg Accuracy: {avg_train_accuracy:.4f}")
print(f"Avg F1 Score: {avg_train_f1:.4f}")
print(f"Avg Kappa: {avg_train_kappa:.4f}")
print(f"Avg AUC: {avg_train_auc:.4f}")
print(f"Avg MCC: {avg_train_mcc:.4f}")

# Calculate average performance metrics across all runs for the test set
avg_test_sensitivity = np.mean(test_sensitivity_list)
avg_test_specificity = np.mean(test_specificity_list)
avg_test_accuracy = np.mean(test_accuracy_list)
avg_test_f1 = np.mean(test_f1_list)
avg_test_kappa = np.mean(test_kappa_list)
avg_test_auc = np.mean(test_auc_list)
avg_test_mcc = np.mean(test_mcc_list)

print("\nTest Set Metrics:")
print(f"Avg Sensitivity: {avg_test_sensitivity:.4f}")
print(f"Avg Specificity: {avg_test_specificity:.4f}")
print(f"Avg Accuracy: {avg_test_accuracy:.4f}")
print(f"Avg F1 Score: {avg_test_f1:.4f}")
print(f"Avg Kappa: {avg_test_kappa:.4f}")
print(f"Avg AUC: {avg_test_auc:.4f}")
print(f"Avg MCC: {avg_test_mcc:.4f}")

feature_names = pd.DataFrame({'Feature Names': selected_features})
# feature_names.to_csv("selected_features.csv", index=False)
print(f"Selected features: {feature_names}")

In [ ]:
# No LASSO
# Load data
train = pd.read_csv("path_to_mRNA_data/input.csv", index_col=0)
target = pd.read_csv("path_to_labels/label.csv", index_col=0).loc[train.columns, :].values.ravel()
indexs = train.index
train = train.T.values
print("train.shape:", train.shape)

# Parameters
loopn = 10  # number of repetitions
n_folds = 10  # 10-fold cross-validation

# Store performance metrics across runs for training and testing sets
train_sensitivity_list, train_specificity_list, train_accuracy_list, train_f1_list, train_kappa_list, train_auc_list, train_mcc_list = [], [], [], [], [], [], []
test_sensitivity_list, test_specificity_list, test_accuracy_list, test_f1_list, test_kappa_list, test_auc_list, test_mcc_list = [], [], [], [], [], [], []

# Perform feature scaling on the entire dataset
standardScaler = StandardScaler()
X_standard = standardScaler.fit_transform(train)

# Run loop for each repetition
np.random.seed(10)
random_states = np.random.choice(range(101), loopn, replace=False)

for i in tqdm(range(loopn)):
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_standard, target, test_size=0.4, random_state=random_states[i])

    # Initialize logistic regression model
    logistic_reg_model = LogisticRegression()

    # Initialize StratifiedKFold for cross-validation on the training set
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=random_states[i])

    # Store performance metrics for each fold on the training set
    fold_train_sensitivity, fold_train_specificity, fold_train_accuracy, fold_train_f1, fold_train_kappa, fold_train_auc, fold_train_mcc = [], [], [], [], [], [], []

    for train_index, val_index in skf.split(X_train, y_train):
        X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
        y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

        # Train the model on the current fold
        logistic_reg_model.fit(X_train_fold, y_train_fold)

        # Predict on the validation set of the current fold
        y_pred_val = logistic_reg_model.predict(X_val_fold)
        y_prob_val = logistic_reg_model.predict_proba(X_val_fold)[:, 1]

        # Calculate confusion matrix for the current fold on the training set
        tn, fp, fn, tp = confusion_matrix(y_val_fold, y_pred_val).ravel()
        fold_train_sensitivity.append(tp / (tp + fn))
        fold_train_specificity.append(tn / (tn + fp))
        fold_train_accuracy.append(accuracy_score(y_val_fold, y_pred_val))
        fold_train_f1.append(f1_score(y_val_fold, y_pred_val))
        fold_train_kappa.append(cohen_kappa_score(y_val_fold, y_pred_val))
        fold_train_auc.append(roc_auc_score(y_val_fold, y_prob_val))
        fold_train_mcc.append(matthews_corrcoef(y_val_fold, y_pred_val))

    # Average performance metrics across folds on the training set for the current run
    train_sensitivity_list.append(np.mean(fold_train_sensitivity))
    train_specificity_list.append(np.mean(fold_train_specificity))
    train_accuracy_list.append(np.mean(fold_train_accuracy))
    train_f1_list.append(np.mean(fold_train_f1))
    train_kappa_list.append(np.mean(fold_train_kappa))
    train_auc_list.append(np.mean(fold_train_auc))
    train_mcc_list.append(np.mean(fold_train_mcc))

    # Evaluate the model on the independent test set
    y_pred_test = logistic_reg_model.predict(X_test)
    y_prob_test = logistic_reg_model.predict_proba(X_test)[:, 1]
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred_test).ravel()

    test_sensitivity_list.append(tp / (tp + fn))
    test_specificity_list.append(tn / (tn + fp))
    test_accuracy_list.append(accuracy_score(y_test, y_pred_test))
    test_f1_list.append(f1_score(y_test, y_pred_test))
    test_kappa_list.append(cohen_kappa_score(y_test, y_pred_test))
    test_auc_list.append(roc_auc_score(y_test, y_prob_test))
    test_mcc_list.append(matthews_corrcoef(y_test, y_pred_test))

# Calculate average performance metrics across runs for the training set
avg_train_sensitivity = np.mean(train_sensitivity_list)
avg_train_specificity = np.mean(train_specificity_list)
avg_train_accuracy = np.mean(train_accuracy_list)
avg_train_f1 = np.mean(train_f1_list)
avg_train_kappa = np.mean(train_kappa_list)
avg_train_auc = np.mean(train_auc_list)
avg_train_mcc = np.mean(train_mcc_list)

print("Training Set Metrics:")
print(f"Avg Sensitivity: {avg_train_sensitivity:.4f}")
print(f"Avg Specificity: {avg_train_specificity:.4f}")
print(f"Avg Accuracy: {avg_train_accuracy:.4f}")
print(f"Avg F1 Score: {avg_train_f1:.4f}")
print(f"Avg Kappa: {avg_train_kappa:.4f}")
print(f"Avg AUC: {avg_train_auc:.4f}")
print(f"Avg MCC: {avg_train_mcc:.4f}")

# Calculate average performance metrics across all runs for the test set
avg_test_sensitivity = np.mean(test_sensitivity_list)
avg_test_specificity = np.mean(test_specificity_list)
avg_test_accuracy = np.mean(test_accuracy_list)
avg_test_f1 = np.mean(test_f1_list)
avg_test_kappa = np.mean(test_kappa_list)
avg_test_auc = np.mean(test_auc_list)
avg_test_mcc = np.mean(test_mcc_list)

print("\nTest Set Metrics:")
print(f"Avg Sensitivity: {avg_test_sensitivity:.4f}")
print(f"Avg Specificity: {avg_test_specificity:.4f}")
print(f"Avg Accuracy: {avg_test_accuracy:.4f}")
print(f"Avg F1 Score: {avg_test_f1:.4f}")
print(f"Avg Kappa: {avg_test_kappa:.4f}")
print(f"Avg AUC: {avg_test_auc:.4f}")
print(f"Avg MCC: {avg_test_mcc:.4f}")
